In [1]:
import os
import sys
sys.path.append(os.getcwd())
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.io.wavfile import read
from glob import glob
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image


2024-11-17 20:57:22.251874: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-17 20:57:22.251949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 20:57:22.319448: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 20:57:22.466253: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-17 20:57:23.816499: W tensorflow/compiler/tf2

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2024-11-17 20:57:26.491989: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 20:57:26.693714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 20:57:26.693783: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 20:57:26.875818: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 20:57:26.875914: I external/local_xla/xla/stream_executor

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16984823822929845217
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10650386432
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13813672627611018968
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [1]:
def preprocess(data_path):
   try:
      array = read(data_path)[1]
      array = array.astype(np.float32)
      if array.ndim >= 2:
         array = np.mean(array, axis=1)
      if array.shape[0] > 96000:
         array = array[:96000]
         return array
      else:
         array = np.append(array, np.zeros(96000-array.shape[0]))
         pass
      return array
   except ValueError:
      pass


data_dir = os.path.join(os.getcwd().rsplit("/",1)[0],"origin_data")
animal_type = "dog"
# state_list = [dir for dir in os.listdir(os.path.join(data_dir,animal_type))]
state_list = ["hostile","relax","whining"]

X_train = []
y_train = []
for state in state_list:
   state_data = glob(os.path.join(data_dir,animal_type,state,"*.wav"))
   for d_path in state_data:
      if not isinstance(preprocess(d_path), np.ndarray):
         continue
      zero = np.zeros(3)
      zero[state_list.index(state)] = 1
      X_train.append(preprocess(d_path))
      y_train.append(zero)
      
X_train = np.array(X_train)
y_train = np.array(y_train)

valid_rate = 0.1
lenX = len(X_train)

random_batch = np.random.choice(range(lenX), int(lenX*valid_rate), replace=False)
random_batch

X_val = X_train[random_batch]
y_val = y_train[random_batch]

X_train = np.delete(X_train, random_batch,axis=0)
y_train = np.delete(y_train, random_batch,axis=0)



NameError: name 'os' is not defined

In [6]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((278, 96000), (278,), (30, 96000), (30,))

In [7]:
from WaveNetClassifier import WaveNetClassifier
# from https://github.com/mjpyeon/wavenet-classifier.git

epoch = 5
wnc = WaveNetClassifier((96000,), (3,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

2024-11-17 20:58:42.159346: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 20:58:42.159536: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 20:58:42.159583: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 20:58:42.159800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-17 20:58:42.159846: I external/local_xla/xla/stream_executor

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 original_input (InputLayer  [(None, 96000)]              0         []                            
 )                                                                                                
                                                                                                  
 reshaped_input (Reshape)    (None, 96000, 1)             0         ['original_input[0][0]']      
                                                                                                  
 dilated_conv_1 (Conv1D)     (None, 96000, 40)            120       ['reshaped_input[0][0]']      
                                                                                                  
 dilated_conv_2_tanh (Conv1  (None, 96000, 40)            3240      ['dilated_conv_1[0][0]']  

In [8]:
wnc.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = epoch, batch_size = 32, optimizer='adam', save=True, save_dir='./')

Epoch 1/5


ValueError: in user code:

    File "/home/ddust/anaconda3/envs/furry/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/ddust/anaconda3/envs/furry/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ddust/anaconda3/envs/furry/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/ddust/anaconda3/envs/furry/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/ddust/anaconda3/envs/furry/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/home/ddust/anaconda3/envs/furry/lib/python3.11/si